In [ ]:
from datetime import datetime
import requests
import pandas as pd
from io import StringIO
from google.colab import files
import numpy as np

# Função para validar a data fornecida pelo usuário
def obter_data():
    while True:
        data_str = input("Digite uma data no formato DD/MM/AAAA (entre 2008 e 2024): ")
        try:
            data = datetime.strptime(data_str, "%d/%m/%Y")
            if 2008 <= data.year <= 2024:
                return data
            else:
                print("Ano fora do intervalo permitido. Tente novamente.")
        except ValueError:
            print("Data inválida. Certifique-se de usar o formato DD/MM/AAAA e que a data existe.")

# Função para validar o horário ou intervalo de horário fornecido pelo usuário
def obter_horario():
    horarios_validos = [0, 6, 12, 18]
    while True:
        try:
            horario_inicial = int(input("Digite o horário inicial ou fixo (0, 6, 12, 18): "))
            if horario_inicial in horarios_validos:
                resposta = input("Deseja fornecer um horário final para criar um intervalo? (s/n): ").lower()
                if resposta == 's':
                    horario_final = int(input("Digite o horário final (0, 6, 12, 18): "))
                    if horario_final in horarios_validos and horario_final >= horario_inicial:
                        return horario_inicial, horario_final
                    else:
                        print("Horário final inválido. Ele deve ser maior ou igual ao horário inicial e ser um dos horários válidos. Tente novamente.")
                else:
                    return horario_inicial, None
            else:
                print("Horário inválido. Escolha entre 0, 6, 12, ou 18.")
        except ValueError:
            print("Entrada inválida. Certifique-se de digitar números inteiros.")

# Função para ajustar longitude para o intervalo de 0 a 360 graus
def ajustar_longitude(lon):
    if lon < 0:
        lon += 360
    return lon

# Função para baixar o arquivo de uma URL
def baixar_arquivo(url):
    try:
        resposta = requests.get(url)
        resposta.raise_for_status()
        return resposta.text
    except requests.HTTPError as e:
        print(f"Erro ao baixar o arquivo: {e}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

# Função para calcular a distância euclidiana entre dois pontos
def calcular_distancia(x1, y1, x2, y2):
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# Função para obter os 4 pontos mais próximos que formam um retângulo ao redor do ponto de interpolação
def obter_pontos_proximos(df, lat_alvo, lon_alvo):
    df_lat_prox = df[(df['lat'] >= lat_alvo - 1) & (df['lat'] <= lat_alvo + 1)]
    df_lon_prox = df_lat_prox[(df_lat_prox['lon'] >= lon_alvo - 1) & (df_lat_prox['lon'] <= lon_alvo + 1)]

    lat_min = df_lon_prox['lat'].min()
    lat_max = df_lon_prox['lat'].max()
    lon_min = df_lon_prox['lon'].min()
    lon_max = df_lon_prox['lon'].max()

    pontos_retangulo = df_lon_prox[
        ((df_lon_prox['lat'] == lat_min) | (df_lon_prox['lat'] == lat_max)) &
        ((df_lon_prox['lon'] == lon_min) | (df_lon_prox['lon'] == lon_max))
    ]

    if len(pontos_retangulo) < 4:
        raise ValueError("Não foi possível encontrar 4 pontos adequados para a interpolação.")

    # Verificar se a latitude e longitude alvo estão dentro dos intervalos encontrados
    if not (lat_min <= lat_alvo <= lat_max and lon_min <= lon_alvo <= lon_max):
        raise ValueError("As coordenadas fornecidas estão fora do intervalo dos pontos disponíveis.")

    print(f"Coordenadas dos 4 pontos usados para interpolação (Lat, Lon):")
    for index, row in pontos_retangulo.iterrows():
        print(f"({row['lat']}, {row['lon']}) -> Distância: {calcular_distancia(row['lat'], row['lon'], lat_alvo, lon_alvo)}")

    return pontos_retangulo

# Função para interpolação bilinear
def bilinear_interpolation(latd, lond, pontos):
    try:
        x1, y1, v1 = pontos.iloc[0]['lon'], pontos.iloc[0]['lat'], pontos.iloc[0]['valor']
        x2, y2, v2 = pontos.iloc[1]['lon'], pontos.iloc[1]['lat'], pontos.iloc[1]['valor']
        x3, y3, v3 = pontos.iloc[2]['lon'], pontos.iloc[2]['lat'], pontos.iloc[2]['valor']
        x4, y4, v4 = pontos.iloc[3]['lon'], pontos.iloc[3]['lat'], pontos.iloc[3]['valor']

        print(f"Pontos para interpolação:")
        print(f"Ponto 1: x1={x1}, y1={y1}, v1={v1}")
        print(f"Ponto 2: x2={x2}, y2={y2}, v2={v2}")
        print(f"Ponto 3: x3={x3}, y3={y3}, v3={v3}")
        print(f"Ponto 4: x4={x4}, y4={y4}, v4={v4}")

        # Calcular ga e gb
        ga = ((lond - x1) * (v2 - v1) / (x4 - x3)) + v1
        gb = ((lond - x1) * (v4 - v3) / (x4 - x3)) + v3

        # Calcular gi
        gi = ((((y3 - y1) - (latd - y1)) * (ga - gb) / (y3 - y1)) + gb)

        print(f"Resultado da interpolação: {gi}")
        return gi
    except IndexError:
        raise ValueError("Erro na seleção dos pontos para interpolação.")

# Função para processar o conteúdo do arquivo e salvar como ACE3
def processar_arquivo(conteudo, caminho_arquivo, lat_alvo, lon_alvo, ano, mes, dia, horario_inicial, horario_final):
    print("Primeiras linhas do arquivo:")
    print(conteudo[:500])

    try:
        linhas = conteudo.splitlines()
        dados_inicio = False
        linhas_dados = []

        for linha in linhas:
            if linha.startswith('!'):
                continue
            if linha.strip() == '':
                continue
            if not dados_inicio:
                if len(linha.split()) >= 10:
                    dados_inicio = True
            if dados_inicio:
                linhas_dados.append(linha)

        dados = StringIO('\n'.join(linhas_dados))
        df = pd.read_csv(dados, delim_whitespace=True, header=None)

        colunas = ['lat', 'lon', 'ah', 'aw', 'zhd', 'zwd', 'Gn_h', 'Ge_h', 'Gn_w', 'Ge_w']
        if len(df.columns) == len(colunas):
            df.columns = colunas

            pontos_proximos = obter_pontos_proximos(df, lat_alvo, lon_alvo)

            precisao = {col: df[col].apply(lambda x: len(str(x).split('.')[1]) if '.' in str(x) else 0).max() for col in colunas[2:]}

            precisao['ah'] = 10
            precisao['aw'] = 10

            valores_interpolados = {}
            for col in colunas[2:]:
                pontos_coluna = pontos_proximos[['lat', 'lon', col]].rename(columns={col: 'valor'})
                try:
                    interpolado = bilinear_interpolation(lat_alvo, lon_alvo, pontos_coluna)
                    if pd.isna(interpolado):
                        print(f"Não foi possível interpolar o valor para a variável {col} nas coordenadas fornecidas.")
                    else:
                        valores_interpolados[col] = interpolado
                except ValueError as e:
                    print(f"Erro na interpolação da variável {col}: {e}")

            with open(caminho_arquivo, 'w') as f:
                f.write(f"ACECOR {ano} {mes:02d} {dia:02d} {horario_inicial:02d} {horario_final if horario_final else horario_inicial:02d}\n")
                f.write("LAT LON AH AW ZHD ZWD GN_H GE_H GN_W GE_W\n")

                f.write(f"{lat_alvo:6.2f} {lon_alvo:7.2f}")
                for col in colunas[2:]:
                    if col in valores_interpolados:
                        f.write(f" {valores_interpolados[col]:.{precisao.get(col, 6)}f}")
                    else:
                        f.write("  NaN")
                f.write("\n")

            print(f"Arquivo ACE3 '{caminho_arquivo}' gerado com sucesso.")
        else:
            print("O formato dos dados no arquivo não corresponde ao esperado.")
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo: {e}")

# Função principal
def main():
    data = obter_data()
    ano, mes, dia = data.year, data.month, data.day

    horario_inicial, horario_final = obter_horario()
    if horario_final is None:
        horarios = [horario_inicial]
    else:
        horarios = list(range(horario_inicial, horario_final + 1, 6))

    latitude = float(input("Digite a latitude de destino: "))
    longitude = float(input("Digite a longitude de destino: "))
    longitude = ajustar_longitude(longitude)

    for horario in horarios:
        url = f"https://vmf.geo.tuwien.ac.at/trop_products/GRID/1x1/V3GR/V3GR_OP/{ano}/V3GR_{ano}{mes:02d}{dia:02d}.H{horario:02d}"
        print(f"URL gerada: {url}")

        conteudo = baixar_arquivo(url)
        if conteudo:
            caminho_arquivo = f"/content/troposfera_{ano}{mes:02d}{dia:02d}_H{horario:02d}_{horario:02d}.ace3"
            processar_arquivo(conteudo, caminho_arquivo, latitude, longitude, ano, mes, dia, horario, horario)

            # Facilitar o download do arquivo para o computador
            files.download(caminho_arquivo)

if __name__ == "__main__":
    main()


Digite uma data no formato DD/MM/AAAA (entre 2008 e 2024): 01/01/2020
Digite o horário inicial ou fixo (0, 6, 12, 18): 0
Deseja fornecer um horário final para criar um intervalo? (s/n): s
Digite o horário final (0, 6, 12, 18): 6
Digite a latitude de destino: -25.441819547393575
Digite a longitude de destino: -49.241947626366546
URL gerada: https://vmf.geo.tuwien.ac.at/trop_products/GRID/1x1/V3GR/V3GR_OP/2020/V3GR_20200101.H00
Primeiras linhas do arquivo:
! Version:            1.0
! Source:             D. Landskron, TU Vienna (created: 2020-01-02)
! Data_types:         V3GR (lat lon ah aw zhd zwd Gn_h Ge_h Gn_w Ge_w)
! Epoch:              2020 01 01 00 00  0.0
! Scale_factor:       1.e+00
! Range/resolution:   -89.5 89.5 0.5 359.5 1 1
! Comment:            vmf.geo.tuwien.ac.at/trop_products/GRID/1x1/V3GR/V3GR_OP/2020/
 89.5    0.5  0.00114369  0.00069101  2.2709  0.0134   0.102   0.270   0.026   0.079
 89.5    1.5  0.00114368  0.00068701  2.2709 
Coordenadas dos 4 pontos usados para int

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

URL gerada: https://vmf.geo.tuwien.ac.at/trop_products/GRID/1x1/V3GR/V3GR_OP/2020/V3GR_20200101.H06
Primeiras linhas do arquivo:
! Version:            1.0
! Source:             D. Landskron, TU Vienna (created: 2020-01-02)
! Data_types:         V3GR (lat lon ah aw zhd zwd Gn_h Ge_h Gn_w Ge_w)
! Epoch:              2020 01 01 06 00  0.0
! Scale_factor:       1.e+00
! Range/resolution:   -89.5 89.5 0.5 359.5 1 1
! Comment:            vmf.geo.tuwien.ac.at/trop_products/GRID/1x1/V3GR/V3GR_OP/2020/
 89.5    0.5  0.00114310  0.00040453  2.2779  0.0109   0.028   0.167   0.084   0.124
 89.5    1.5  0.00114311  0.00039751  2.2779 
Coordenadas dos 4 pontos usados para interpolação (Lat, Lon):
(-24.5, 310.5) -> Distância: 0.976532174273076
(-24.5, 311.5) -> Distância: 1.198962192950044
(-25.5, 310.5) -> Distância: 0.26452975750064867
(-25.5, 311.5) -> Distância: 0.7442252651828138
Pontos para interpolação:
Ponto 1: x1=310.5, y1=-24.5, v1=0.00127557
Ponto 2: x2=311.5, y2=-24.5, v2=0.00127514
Ponto

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>